**Table of contents**<a id='toc0_'></a>    
- [<b>Import Libraries</b>](#toc1_)    
- [<b>1. Load Dataset</b>](#toc2_)    
  - [Load Configuration File](#toc2_1_)    
  - [Data Collection](#toc2_2_)    
- [<b>2. Data Validation </b>](#toc3_)    
  - [Types of Data](#toc3_1_)    
  - [Data Range](#toc3_2_)    
  - [<b>Data Dimension </b>](#toc3_3_)    
- [<b>Handling Columns Type</b>](#toc4_)    
  - [Check Dataset Type](#toc4_1_)    
- [<b>Handling data category</b>](#toc5_)   
<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[<b>Import Libraries</b>](#toc0_)

In [19]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os
import yaml
import copy
import src.util as util
import warnings
warnings.filterwarnings('ignore')

# <a id='toc2_'></a>[<b>1. Load Dataset</b>](#toc0_)

In [5]:
dataset = pd.read_csv(r"insurance_claims.csv")

In [6]:
dataset

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,_c39
0,328,48,521585,2014-10-17,OH,250/500,1000,1406.91,0,466132,...,YES,71610,6510,13020,52080,Saab,92x,2004,Y,NaN
1,228,42,342868,2006-06-27,IN,250/500,2000,1197.22,5000000,468176,...,?,5070,780,780,3510,Mercedes,E400,2007,Y,NaN
2,134,29,687698,2000-09-06,OH,100/300,2000,1413.14,5000000,430632,...,NO,34650,7700,3850,23100,Dodge,RAM,2007,N,NaN
3,256,41,227811,1990-05-25,IL,250/500,2000,1415.74,6000000,608117,...,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2014,Y,NaN
4,228,44,367455,2014-06-06,IL,500/1000,1000,1583.91,6000000,610706,...,NO,6500,1300,650,4550,Accura,RSX,2009,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3,38,941851,1991-07-16,OH,500/1000,1000,1310.80,0,431289,...,?,87200,17440,8720,61040,Honda,Accord,2006,N,NaN
996,285,41,186934,2014-01-05,IL,100/300,1000,1436.79,0,608177,...,?,108480,18080,18080,72320,Volkswagen,Passat,2015,N,NaN
997,130,34,918516,2003-02-17,OH,250/500,500,1383.49,3000000,442797,...,YES,67500,7500,7500,52500,Suburu,Impreza,1996,N,NaN
998,458,62,533940,2011-11-18,IL,500/1000,2000,1356.92,5000000,441714,...,YES,46980,5220,5220,36540,Audi,A5,1998,N,NaN


# <a id='toc3_'></a>[<b>2. Data Validation </b>](#toc0_)

## <a id='toc3_1_'></a>[Types of Data](#toc0_)

In [8]:
dataset.dtypes

months_as_customer               int64
age                              int64
policy_number                    int64
policy_bind_date                object
policy_state                    object
policy_csl                      object
policy_deductable                int64
policy_annual_premium          float64
umbrella_limit                   int64
insured_zip                      int64
insured_sex                     object
insured_education_level         object
insured_occupation              object
insured_hobbies                 object
insured_relationship            object
capital-gains                    int64
capital-loss                     int64
incident_date                   object
incident_type                   object
collision_type                  object
incident_severity               object
authorities_contacted           object
incident_state                  object
incident_city                   object
incident_location               object
incident_hour_of_the_day 

## <a id='toc3_2_'></a>[Data Range](#toc0_)

In [10]:
# some columns have minimum value of zero, this value has been encoded as a missing value 
# instead of None Value or Misig cell.
dataset.describe()

,months_as_customer,age,policy_number,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,capital-gains,capital-loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_year,_c39
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1.000000e+03,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,0.0
mean,203.954000,38.948000,546238.648000,1136.000000,1256.406150,1.101000e+06,501214.488000,25126.100000,-26793.700000,11.644000,1.83900,0.992000,1.487000,52761.94000,7433.420000,7399.570000,37928.950000,2005.103000,NaN
std,115.113174,9.140287,257063.005276,611.864673,244.167395,2.297407e+06,71701.610941,27872.187708,28104.096686,6.951373,1.01888,0.820127,1.111335,26401.53319,4880.951853,4824.726179,18886.252893,6.015861,NaN
min,0.000000,19.000000,100804.000000,500.000000,433.330000,-1.000000e+06,430104.000000,0.000000,-111100.000000,0.000000,1.00000,0.000000,0.000000,100.00000,0.000000,0.000000,70.000000,1995.000000,NaN
25%,115.750000,32.000000,335980.250000,500.000000,1089.607500,0.000000e+00,448404.500000,0.000000,-51500.000000,6.000000,1.00000,0.000000,1.000000,41812.50000,4295.000000,4445.000000,30292.500000,2000.000000,NaN
50%,199.500000,38.000000,533135.000000,1000.000000,1257.200000,0.000000e+00,466445.500000,0.000000,-23250.000000,12.000000,1.00000,1.000000,1.000000,58055.00000,6775.000000,6750.000000,42100.000000,2005.000000,NaN
75%,276.250000,44.000000,759099.750000,2000.000000,1415.695000,0.000000e+00,603251.000000,51025.000000,0.000000,17.000000,3.00000,2.000000,2.000000,70592.50000,11305.000000,10885.000000,50822.500000,2010.000000,NaN
max,479.000000,64.000000,999435.000000,2000.000000,2047.590000,1.000000e+07,620962.000000,100500.000000,0.000000,23.000000,4.00000,2.000000,3.000000,114920.00000,21450.000000,23670.000000,79560.000000,2015.000000,NaN


## <a id='toc3_3_'></a>[<b>Data Dimension </b>](#toc0_)

In [11]:
dataset.shape

(1000, 40)

# <a id='toc4_'></a>[<b>Handling Columns Type</b>](#toc0_)

Change <b>Capital-gains</b> and <b>capital-loss</b> column name

In [12]:
dataset =dataset.rename(columns = {'capital-gains':'capital_gains'})
dataset =dataset.rename(columns = {'capital-loss':'capital_loss'})

Convert dataset <i>datetime</i>, <br>
convert data type <b>float</b> into <b>int32</b>, <br> 
convert data type <b>int64</b> into <b>int32</b> and <br>
convert few of data type <b>int</b> into <b>object</b> 

In [21]:
config_data = util.load_config()

In [22]:
def type_data(set_data):
    """Change raw dataset type.

    Convert raw dataset datetime, float into int, int64 to int32 and convert int columns into object
    """
    # --- Change datetime object ---#
    for col in config_data["datetime_columns"]:
        set_data[col] = pd.to_datetime(set_data[col])

    # --- Columns _c39 is not defined columns, then drop the columns -- #
    set_data = set_data.drop(columns = ['_c39'], axis=1)

    # --- change columns type policy_annual_premium columns into int64 ---#
    set_data = set_data.astype({col: 'int32' for col in set_data.select_dtypes('float64').columns})

    # --- convert int64 into int32 --- #
    set_data = set_data.astype({col: 'int32' for col in set_data.select_dtypes('int64').columns})

    # --- convert few of int columns into object --- #
    raw_dataset_date = set_data[config_data['datetime_columns']]
    raw_dataset_num = set_data[config_data['int32_col']]

    list_of_non_cat = config_data['int32_col'].copy()
    list_of_non_cat = list_of_non_cat + config_data['datetime_columns']

    raw_dataset_cat = set_data[config_data['object_columns']]
    raw_dataset_cat = raw_dataset_cat.astype(str)

    # --- Return raw dataset with datetime, numerical int, and category object --#
    raw_dataset = pd.concat([raw_dataset_date, raw_dataset_num, raw_dataset_cat], axis = 1)
    
    # --- Handle umbrella limit data --- #
    raw_dataset = raw_dataset.replace({'umbrella_limit': '-1000000'}, '1000000') 

    return raw_dataset

In [23]:
raw_dataset = type_data(dataset)

## <a id='toc4_1_'></a>[Check Dataset Type](#toc0_)

In [24]:
## Check dataset type of each features
raw_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   policy_bind_date             1000 non-null   datetime64[ns]
 1   incident_date                1000 non-null   datetime64[ns]
 2   months_as_customer           1000 non-null   int32         
 3   age                          1000 non-null   int32         
 4   policy_number                1000 non-null   int32         
 5   policy_annual_premium        1000 non-null   int32         
 6   insured_zip                  1000 non-null   int32         
 7   capital_gains                1000 non-null   int32         
 8   capital_loss                 1000 non-null   int32         
 9   incident_hour_of_the_day     1000 non-null   int32         
 10  total_claim_amount           1000 non-null   int32         
 11  injury_claim                 1000 non-null  

In [25]:
## Sanity Check
raw_dataset.shape

(1000, 35)

# <a id='toc5_'></a>[<b>Handling data category</b>](#toc0_)

In [26]:
## Check categorical data
pd.set_option('display.max_rows', None)

pd.DataFrame(
        raw_dataset[raw_dataset.select_dtypes('object').columns.to_list()]
        .melt(var_name='columns', value_name='index')\
        .value_counts())\
        .sort_values(by=['columns',0])

0
columns                     index                        
authorities_contacted       None                       91
                            Ambulance                 196
                            Other                     198
                            Fire                      223
                            Police                    292
auto_make                   Honda                      55
                            Mercedes                   65
                            Jeep                       67
                            Volkswagen                 68
                            Accura                     68
                            Audi                       69
                            Toyota                     70
                            Ford                       72
                            BMW                        72
                            Chevrolet                  76
                            Nissan                     78
                            Dodge                      80
                            Suburu                     80
                            Saab                       80
auto_model                  RSX                        12
                            Accord                     13
                            M5                         15
                            X6                         16
                            3 Series                   18
                            C300                       18
                            Impreza                    20
                            CRV                        20
                            TL                         20
                            ML350                      20
                            Corolla                    20
                            Fusion                     21
                            Silverado                  22
                            Highlander                 22
                            Civic                      22
                            X5                         23
                            Ultima                     23
                            Maxima                     24
                            Escape                     24
                            Tahoe                      24
                            93                         25
                            Grand Cherokee             25
                            95                         27
                            E400                       27
                            F150                       27
                            Forrestor                  28
                            92x                        28
                            Camry                      28
                            Malibu                     30
                            Pathfinder                 31
                            A5                         32
                            Legacy                     32
                            Passat                     33
                            Jetta                      35
                            MDX                        36
                            Neon                       37
                            A3                         37
                            Wrangler                   42
                            RAM                        43
auto_year                   1996                       37
                            2004                       39
                            1998                       40
                            2001                       42
                            2000                       42
                            2014                       44
                            2008                       45
                            1997                       46
                            2012                       46
                            2015                      

In [27]:
# --- Change ? category value into UNKNOWN
raw_dataset.collision_type = raw_dataset.collision_type.replace("?","UNKNOWN")
raw_dataset.property_damage = raw_dataset.property_damage.replace("?", "UNKNOWN")
raw_dataset.police_report_available = raw_dataset.police_report_available.replace("?", "UNKNOWN")
raw_dataset.authorities_contacted = raw_dataset.authorities_contacted.replace("None", "UNKNOWN")

In [28]:
# Check label data
raw_dataset.fraud_reported.value_counts(normalize=True)

N    0.753
Y    0.247
Name: fraud_reported, dtype: float64

In [40]:
raw_dataset

,policy_bind_date,incident_date,months_as_customer,age,policy_number,policy_annual_premium,insured_zip,capital_gains,capital_loss,incident_hour_of_the_day,...,incident_city,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,auto_year,police_report_available,auto_make,auto_model,fraud_reported
0,2014-10-17,2015-01-25,328,48,521585,1406,466132,53300,0,5,...,Columbus,1,YES,1,2,2004,YES,Saab,92x,Y
1,2006-06-27,2015-01-21,228,42,342868,1197,468176,0,0,8,...,Riverwood,1,UNKNOWN,0,0,2007,UNKNOWN,Mercedes,E400,Y
2,2000-09-06,2015-02-22,134,29,687698,1413,430632,35100,0,7,...,Columbus,3,NO,2,3,2007,NO,Dodge,RAM,N
3,1990-05-25,2015-01-10,256,41,227811,1415,608117,48900,-62400,5,...,Arlington,1,UNKNOWN,1,2,2014,NO,Chevrolet,Tahoe,Y
4,2014-06-06,2015-02-17,228,44,367455,1583,610706,66000,-46000,20,...,Arlington,1,NO,0,1,2009,NO,Accura,RSX,N
5,2006-10-12,2015-01-02,256,39,104594,1351,478456,0,0,19,...,Arlington,3,NO,0,2,2003,NO,Saab,95,Y
6,2000-06-04,2015-01-13,137,34,413978,1333,441716,0,-77000,0,...,Springfield,3,UNKNOWN,0,0,2012,UNKNOWN,Nissan,Pathfinder,N
7,1990-02-03,2015-02-27,165,37,429027,1137,603195,0,0,23,...,Columbus,3,UNKNOWN,2,2,2015,YES,Audi,A5,N
8,1997-02-05,2015-01-30,27,33,485665,1442,601734,0,0,21,...,Arlington,1,NO,1,1,2012,YES,Toyota,Camry,N
9,2011-07-25,2015-01-05,212,42,636550,1315,600983,0,-39300,14,...,Hillsdale,1,NO,2,1,1996,UNKNOWN,Saab,92x,N


In [ ]:
raw_dataset.to_excel(data_)